In [1]:
example1 = """
root: pppw + sjmn
dbpl: 5
cczh: sllz + lgvd
zczc: 2
ptdq: humn - dvpt
dvpt: 3
lfqf: 4
humn: 5
ljgn: 2
sjmn: drzm * dbpl
sllz: 4
pppw: cczh / lfqf
lgvd: ljgn * ptdq
drzm: hmdt - zczc
hmdt: 32
"""
from collections import defaultdict
from dataclasses import dataclass
from pprint import pprint

data = open("input.txt").read()

In [2]:
@dataclass
class Eq:
    a: str | int
    op: str
    b: str | int

    def set_(self, id_, n):
        if self.a == id_:
            self.a = n
        if self.b == id_:
            self.b = n

    def can_eval(self):
        return isinstance(self.a, (int, float)) and isinstance(self.b, (int, float))

    def eval(self):
        return int(eval(f"{self.a} {self.op} {self.b}"))


def task1(data):
    numbers = {}
    eqs = defaultdict(list)

    stack = []
    for line in data.strip().splitlines():
        id_, eq = line.split(": ")
        try:
            n = int(eq)
        except ValueError:
            id2, op, id3 = eq.split()
            eq = Eq(id2, op, id3)
            eqs[id2].append((id_, eq))
            eqs[id3].append((id_, eq))
        else:
            stack.append((id_, n))

    while stack:
        id_, n = stack.pop()
        numbers[id_] = n
        id_eqs = eqs[id_]
        for eq_id, eq in id_eqs:
            if eq_id in numbers:
                continue
            eq.set_(id_, n)
            if eq.can_eval():
                try:
                    stack.append((eq_id, eq.eval()))
                except ZeroDivisionError:
                    continue
        id_eqs.clear()

    return numbers["root"]


def task2(data):
    target = "humn"
    numbers = {}
    eqs = defaultdict(list)

    stack = []
    for line in data.strip().splitlines():
        id_, eq = line.split(": ")
        if id_ == "humn":
            continue
        elif id_ == "root":
            id2, op, id3 = eq.split()
            eqs[id3].append((id2, Eq(id3, "+", 0)))
            eqs[id2].append((id3, Eq(id2, "+", 0)))
            continue
        try:
            n = int(eq)
        except ValueError:
            id2, op, id3 = eq.split()
            eq = Eq(id2, op, id3)
            eqs[id2].append((id_, eq))
            eqs[id3].append((id_, eq))

            rop = {
                "+": "-",
                "*": "/",
                "/": "*",
                "-": "+",
            }[op]

            # id_ = id2 / id3
            # id2 = id_*id3
            # id3 = id2 /id_

            # id = id2 * id3
            # id/id3 = id2
            # id/id2 = id3

            # id = id2 + id3
            # id2 = id-id3
            # id3 = id-id2

            # id = id2 - id3
            # id2 = id+id3
            # id3 = id2-id3

            # id_ = id2 op id3
            # id_ rop id3 = id2
            eq2 = Eq(id_, rop, id3)
            eqs[id_].append((id2, eq2))
            eqs[id3].append((id2, eq2))

            # id_ - id2 = id3
            if op == "-":
                eq3 = Eq(id2, "-", id_)
            else:
                if op == "/":
                    rop = "/"
                eq3 = Eq(id_, rop, id2)
            eqs[id_].append((id3, eq3))
            eqs[id2].append((id3, eq3))
        else:
            stack.append((id_, n))

    while stack:
        id_, n = stack.pop()
        numbers[id_] = n
        id_eqs = eqs[id_]
        for eq_id, eq in id_eqs:
            eq.set_(id_, n)
            if eq.can_eval():
                if eq_id == target:
                    return eq.eval()
                # print(eq_id, eq)
                try:
                    stack.append((eq_id, eq.eval()))
                except ZeroDivisionError:
                    continue
        # id_eqs[:] = [(eq_id, eq)  for (eq_id, eq) in id_eqs if not eq.can_eval()]
        id_eqs.clear()
    pprint({id_: id_eqs for id_, id_eqs in eqs.items() if id_eqs})
    return numbers["humn"]


print(f"{task1(example1)=}")
print(f"{task1(data)=}")
print(f"{task2(example1)=}")
print(f"{task2(data)=}")

task1(example1)=152
task1(data)=379578518396784
task2(example1)=301
task2(data)=3353687996514
